In [1]:
"""
Compute the required kernel quantities

* \$\$K^{n}_{tu} \\in \\mathcal{R}^{\\sum_m\\mathcal{T}_m\\times\\sum_d J_d}\$\$

* \$\$K^{n}_{<t>u} \\in \\mathcal{R}^{M\\times\\sum_d J_d}\$\$

* \$\$K_{uu} \\in \\mathcal{R}^{M\\times\\sum_d J_d}\$\$

and their derivatives
"""
function build_kernels(data::AbstractArray, θ::Union{PoissonProcessParams, AbstractArray}, 
    KernList::AbstractArray; compute_Ktu=false)
    
    C, α, u, N, M, D, J = name_params(θ)
    
    
    # Store the kernels
    Ktu = Array(Any,N) # This is never actually needed for score-matching (compute either this, or the derivatives)
    dtKtu = Array(Any,N) # Score-matching
    ddtKtu = Array(Any,N) # Score-matching
    Kuu = Array(Any,N) # Regularisation term
    
    # Count inducing points
    Jtot = convert(Integer, sum(J))
    
    # Store spike counts
    I = Array(Any, N)
    
    for n = 1:N
        
        I[n] = Array(Integer, M) # Stores spike counts for neuron m on trial n
        # Count spikes
        for m = 1:M
            I[n][m] = convert(Integer,length(data[n][m]))
        end
        Itot = convert(Integer,sum(I[n][:]))


        #Initialise kernel matrices and derivatives
        if compute_Ktu 
            cur_Ktu = zeros(Itot,Jtot) 
        else
            cur_dtKtu = zeros(Itot,Jtot)
            cur_ddtKtu = zeros(Itot,Jtot)
        end


        cur_Kuu = zeros(Jtot,Jtot)

        Tn = zeros(M)
        for d = 1:D
            d_inds = (d>1?cumsum(J)[d-1]+1:1):cumsum(J)[d];

            # Compute kernel derivative functions wrt t given the u
            if compute_Ktu
                orig_t = [t->kernel(KernList[d], t, u1) for u1 in u[d]]
            else
                dt = [Calculus.derivative(t->kernel(KernList[d], t, u1)) for u1 in u[d]]
                ddt = [Calculus.second_derivative(t->kernel(KernList[d], t, u1)) for u1 in u[d]]
            end
            
            # Compute values at the data points
            for m = 1:M
                m_inds = (m>1?cumsum(I[n])[m-1]+1:1):cumsum(I[n])[m];
                for i = 1:I[n][m]
                    if compute_Ktu
                        cur_Ktu[m_inds[i],d_inds] = [t1(data[n][m][i]) for t1 in orig_t]
                    else
                        cur_dtKtu[m_inds[i],d_inds] = [dt1(data[n][m][i]) for dt1 in dt]
                        cur_ddtKtu[m_inds[i],d_inds] = [ddt1(data[n][m][i]) for ddt1 in ddt]
                    end
                end
            end

            cur_Kuu[d_inds, d_inds] = 
                kernelmatrix(KernList[d],reshape(u[d],J[d],1))
        end
        
        if compute_Ktu
            Ktu[n] = cur_Ktu
        else
            dtKtu[n] = cur_dtKtu
            ddtKtu[n] = cur_ddtKtu
        end
        Kuu[n] = cur_Kuu
    end
    
            
    return Ktu, dtKtu, ddtKtu, Kuu, I
    
end

LoadError: LoadError: UndefVarError: PoissonProcessParams not defined
while loading In[1], in expression starting on line 433